In [124]:
import torch.nn as nn
import torch
import numpy as np
import torch.optim as optim
import pandas as pd
from scipy.signal import savgol_filter
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

In [133]:
def getStates():
    ds = pd.read_csv('data/test.csv')
    states = ds['Province_State'][:50].values   
    return states

In [134]:
degree = 3
states = getStates()
start = 132
alpha = .1
window = 17

In [148]:
feature = 'Deaths'
#feature = 'Confirmed'
res = []
for i in range(len(states)):
    state = states[i]
    ds = pd.read_csv('data/train.csv')
    ds = ds[ds['Province_State'] == state]
    value = ds[feature].values
    
    value = savgol_filter(value.reshape(-1), window, degree) #smooth data
    
    combine = [[value[i]] for i in range(len(value))]
    
    y = np.array([value[start:]]).reshape(-1, 1)
    x = [[i + start] for i in range(142-start)]
    
#     model = pl.make_pipeline(
#         sp.PolynomialFeatures(1), 
#         lm.LinearRegression()  
#     )
    model = Lasso(alpha=.05, fit_intercept=True)

#    model = Pipeline([
#        ("poly", PolynomialFeatures(degree=2)),
#        ("lasso_reg", Ridge(alpha=alpha))
#    ])
    
    model.fit(x, y)
    
    x_test = [[i + 142] for i in range(26)]
    
    y_hat = model.predict(x_test)
    res.append(y_hat)
print(res[0])
print(len(res))

[2198.16160991 2219.50855615 2240.85550239 2262.20244863 2283.54939488
 2304.89634112 2326.24328736 2347.59023361 2368.93717985 2390.28412609
 2411.63107233 2432.97801858 2454.32496482 2475.67191106 2497.0188573
 2518.36580355 2539.71274979 2561.05969603 2582.40664227 2603.75358852
 2625.10053476 2646.447481   2667.79442724 2689.14137349 2710.48831973
 2731.83526597]
50


In [136]:
rerange = []
for i in range(len(res[0])):
    for j in range(len(res)):
        rerange.append(res[j][i])
#print(rerange)
df = pd.DataFrame(rerange)
df.to_csv("{}.csv".format(feature),index=False,sep=',')

In [137]:
confirmed_test = pd.read_csv('Confirmed.csv')
real = pd.read_csv('data/test.csv')
deaths_test = pd.read_csv('deaths.csv')

In [144]:
confirmed = confirmed_test['0'].values
deaths = deaths_test['0'].values
print(confirmed)
print(len(confirmed))

[127582.7624355    5353.56357069 202575.34788442 ...  13409.27107609
  93964.85559621   4790.84280889]
1300


In [142]:
real_confirmed = real['Confirmed'].values
real_deaths = real['Deaths'].values
print(real_confirmed)
print(len(real_confirmed))

[127616   5297 202342 ...  15167 113645   5465]
1300


In [140]:
def score():
    total_c = 0
    total_d = 0
    for i in range(len(confirmed)):
        n_c = abs(confirmed[i] - real_confirmed[i]) / real_confirmed[i]
        n_d = abs(deaths[i] - real_deaths[i]) / real_deaths[i]
        total_d += n_d
        total_c += n_c
    
    total_c = total_c * 100 / (1 * len(confirmed))
    total_d = total_d * 100 / (1 * len(deaths))
    print(total_c,total_d)
score()

2.0789093976982924 2.6980373273185636


In [162]:
def stScore():
    for j in range(50):
        total_c = 0
        total_d = 0
        for i in range(26):
            n_c = abs(confirmed[j+50*i] - real_confirmed[j + 50*i]) / real_confirmed[j + 50*i]
            n_d = abs(deaths[j + 50*i] - real_deaths[j + 50*i]) / real_deaths[j + 50*i]
            total_d += n_d
            total_c += n_c
            #print(confirmed[j+50*i], " ",real_confirmed[j + 50*i])
        total_c = total_c * 100 / 26
        total_d = total_d * 100 / 26
        print(states[j]+"        ", total_c, "        ", total_d)
        
stScore()

Alabama         3.811042284081161          4.366080417382475
Alaska         1.2615842009225928          7.334657747786442
Arizona         0.4150094819902818          3.8669501417596686
Arkansas         1.6486219870271563          4.90466036407699
California         2.629239043037003          1.1462719806722002
Colorado         1.2290184009271745          0.15889372536065263
Connecticut         0.4643288003035828          0.19909640452916147
Delaware         2.762816712098487          0.7476042246938969
Florida         0.3964970328458267          0.5675658351501053
Georgia         1.7795464965473233          3.734199413502464
Hawaii         11.539307072374413          3.0562889678607084
Idaho         0.908347764370357          8.692605619714582
Illinois         0.40497726703625675          0.5435369830673562
Indiana         1.921718815525536          0.35421687463468066
Iowa         6.7414607788928445          2.6610265423282744
Kansas         1.8207110654985823          8.4528633392982